
# Using Astropy

## Coordinate conversion

Astropy provides useful tools to convert between the coordinate systems. 

The International Celestial Reference System (ICRS) is the current standard celestial reference system adopted by the International Astronomical Union (IAU).
Its origin is at the barycenter of the Solar System, with axes that are intended to "show no global rotation with respect to a set of distant extragalactic objects". 

In [ ]:
import astropy 
import pylab as pl 
from astropy import units as u
from astropy import constants as const 
from astropy.coordinates import SkyCoord

In [ ]:

coord1 = SkyCoord(ra=10.625*u.degree, dec=41.2*u.degree, frame='icrs')


sometimes however RA is referred in _hourangle_ units and Declination in degrees, minutes and seconds 

In [ ]:
coord2 = SkyCoord('00h42m30s', '+41d12m00s', frame='icrs')

we can then use the conversion tool to switch between one unit frame and the other , as well as different coordinate system

In [ ]:
print(coord1.ra.to((u.hourangle)),
coord2.dec.to(u.deg) ,
coord1.galactic,   )

It is also possible to input coordinate values in other representations
such as cartesian or cylindrical.  In this case one includes the keyword
argument ``representation_type='cartesian'`` (for example) along with data
in ``x``, ``y``, and ``z`` , we need to provide a distance though. this is very useful when dealing with 3D volume data. 

In [ ]:
coord3 = SkyCoord(0*u.deg, 0*u.deg,'100kpc', frame='galactic')
print(coord3, coord3.cartesian, coord3.cylindrical, coord3.spherical) 

## using constants and units 

In [ ]:
Tsun= 5000 *u.K 
print((const.k_B*Tsun).to(u.eV)) 
wavelength= 800 *u.um 
freq= (const.c/wavelength).to(u.THz)
print(freq)

## reading   data FITS format   
Flexible Image Transport System (FITS) is an open standard defining a digital file format useful for storage, transmission and processing of data: formatted as multi-dimensional arrays (for example a 2D image), or tables. FITS is the most commonly used digital file format in astronomy. The FITS standard was designed specifically for astronomical data, and includes provisions such as describing photometric and spatial calibration information, together with image origin metadata.

The FITS format was first standardized in 1981; it has evolved gradually since then, and the most recent version (4.0) was standardized in 2016. FITS was designed with an eye towards long-term archival storage, and the maxim once FITS, always FITS represents the requirement that developments to the format must be backward compatible.

Image metadata is stored in a human-readable ASCII header. The information in this header is designed to calculate the byte offset of some information in the subsequent data unit to support direct access to the data cells. Each FITS file consists of one or more headers containing ASCII card images that carry keyword/value pairs, interleaved between data blocks. The keyword/value pairs provide information such as size, origin, coordinates, binary data format, free-form comments, history of the data, and anything else the creator desires: while many keywords are reserved for FITS use, the standard allows arbitrary use of the rest of the name-space.

FITS is also often used to store non-image data, such as spectra, photon lists, data cubes, or structured data such as multi-table databases. A FITS file may contain several extensions, and each of these may contain a data object. For example, it is possible to store x-ray and infrared exposures in the same file.

#### Download the Data Release 1 of Gaia 

we download  data from the 1st GAIA data release, these data encode parallaxes of tons of stars. 


In [ ]:
from astropy.io import fits ,ascii 
try : 
    hdu = fits.open( 'GaiaSource_000-000-000.fits')
except FileNotFoundError:
    !wget http://cdn.gea.esac.esa.int/Gaia/gdr1/gaia_source/fits/GaiaSource_000-000-000.fits 
    hdu = fits.open( 'GaiaSource_000-000-000.fits')
        

### FITS  headers 
Every Header Data Unit (HDU) normally has two components: header and data. In astropy these two components are accessed through the two attributes of the HDU, hdu.header and hdu.data.

While an HDU may have empty data (i.e., the .data attribute is None), any HDU will always have a header.

Usually the primary header looks like this : 

In [ ]:
hdu[0].header, 

the second header is the card encoding the astrophysical infos, the format of each columns,the units, the name ,etc... 

In [ ]:
hdu[1].header

Astropy provides utils to access the columns entity 

In [ ]:
hdu[1].columns 

In [ ]:
hdu[1].columns['parallax']

whereas the data are accessible with

In [ ]:
hdu[1].data['parallax']

let's see what we can do with data... 

In [ ]:
import numpy as np 
masknans =np.isnan(hdu[1].data['parallax'])
gaia_parallaxes = hdu[1].data['parallax'][~masknans]
min_para= gaia_parallaxes.min() 
max_para= gaia_parallaxes.max() 
pl.figure(figsize=(15,12))
pl.subplot(221)
_=pl.hist(gaia_parallaxes, bins= np.linspace((min_para), (max_para), 50 ), density=True) 

pl.xlim(0,20)
pl.title( hdu[1].columns['parallax'].name ,  fontsize=15)
pl.xlabel(f"{hdu[1].columns['parallax'].unit}", fontsize=12)
pl.subplot(224)
pl.title( hdu[1].columns['ra'].name ,  fontsize=15)
pl.xlabel(f"{hdu[1].columns['ra'].unit}", fontsize=12)
_=pl.hist(hdu[1].data['ra'][~masknans], bins=  30, density=True ) 
pl.subplot(223)
pl.title( hdu[1].columns['dec'].name ,  fontsize=15)
pl.xlabel(f"{hdu[1].columns['dec'].unit}", fontsize=12)
_=pl.hist(hdu[1].data['dec'][~masknans], bins=  30, density=True ) 
pl.subplot(222)
pl.title( hdu[1].columns['phot_g_mean_mag'].name ,  fontsize=15)
pl.xlabel(f"{hdu[1].columns['phot_g_mean_mag'].unit}", fontsize=12)
_=pl.hist(hdu[1].data['phot_g_mean_mag'][~masknans], bins=  30, density=True ) 



In [ ]:
ra_fits=hdu[1].data['ra'][~masknans].copy()
dec_fits=hdu[1].data['dec'][~masknans].copy()
hdu.close() 

always remember to close the HDU, otherwise the file remains open. 


## reading astropy table  
Another format catalogs can be in is `.csv` or `.dat`, in general txt files. Astropy provides a way to read'em and easily access as they were `.fits`.  This is thanks to the [astropy.ascii.table](https://docs.astropy.org/en/stable/table/index.html#module-astropy.table)  

### Gaia Radial Velocity Spectrometer


We download the spectra obtained for several sources acquired with the RVS. 
https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_spectroscopic_tables/ssec_dm_rvs_mean_spectrum.html


In [ ]:
try :
    table = ascii.read('RvsMeanSpectrum_000000-003111.csv.gz' ,format='ecsv' )
except FileNotFoundError:
    !wget http://cdn.gea.esac.esa.int/Gaia/gdr3/Spectroscopy/rvs_mean_spectrum/RvsMeanSpectrum_000000-003111.csv.gz
    table = ascii.read('RvsMeanSpectrum_000000-003111.csv.gz' ,format='ecsv' )


tables are as easy to handle as the fits files, 

In [ ]:
table.colnames, table.columns 

the nicest feature is to print a table 

In [ ]:
table

In [ ]:
nrows=table.as_array().shape[0] 
rvs_bins  = np.linspace(846,870, spectrum.size )

In [ ]:
pl.figure(figsize=(15,12))

for i in range(nrows ):  
    pl.subplot(2,4,i+1 )
    spectrum =   np.float_(table['flux'][i][1:-1].split(',' )  ) 
    pl.title(f"{table['ra'][i]:.2f},{ table['dec'][i]:.2f}")
    pl.plot(rvs_bins,  spectrum ,   )
    if i==7: break 


In [ ]:

ra_table= table['ra'] 
dec_table= table['dec'] 

coords_parallax = SkyCoord(ra=ra_fits *u.degree, dec=dec_fits*u.degree)
coords_spectrum  = SkyCoord(ra=ra_table, dec=dec_table)
#

###  matching  two catalogs

we match  sources in both catalogs  at about  the same position, so that the match results into sources whose we know parallax and spectrum. 
astropy has a routine that does this 


In [ ]:
from astropy.coordinates import match_coordinates_sky
match_coordinates_sky?

In [ ]:
idx, d2d, _ = match_coordinates_sky(coords_parallax , coords_spectrum )

In [ ]:
matchmask = d2d< .1*u.arcsec

In [ ]:
print(f"we matched {coords_parallax[matchmask].size} stars! ") 

In [ ]:
table[idx[matchmask]] 

### write a table and save it 

let's save into disk a new table encoding only the matched sources, we want to include parallax too into this table.


In [ ]:
tab_match= table[idx[matchmask]]

tab_match.add_column(gaia_parallaxes[matchmask] , name='parallax') 

ascii.write(table = tab_match ,names= tab_match.colnames, output=  f'matched_gaiadata.dat', overwrite=True )

##  Cosmology

Another useful feature of Astropy is related to [cosmological calculations](https://docs.astropy.org/en/stable/cosmology/index.html).  
The `astropy.cosmology` sub-package contains classes for representing cosmologies and utility functions for calculating commonly used quantities that depend on a cosmological model. This includes distances, ages, and lookback times corresponding to a measured redshift or the transverse separation corresponding to a measured angular separation.

astropy.cosmology.units extends the astropy.units sub-package, adding and collecting cosmological units and equivalencies, like  for keeping track of (dimensionless) factors of the Hubble constant.

In [ ]:
from astropy.cosmology import Planck18 as cosmo,z_at_value
from astropy.cosmology import FlatLambdaCDM  
cosmo2= FlatLambdaCDM(Ob0=0,Om0=1, H0=70)

In [ ]:
z= np.logspace(-3, 2, 100)
a=  cosmo.scale_factor(z)

In [ ]:
print(cosmo2.age(0),cosmo.age(0))

In [ ]:
pl.figure(figsize=(15,12))
pl.subplot(221)
pl.loglog(z, cosmo.age(z) , label='Age' ) 
pl.loglog(z, cosmo.lookback_time(z)  , label='Lookback time') 
pl.legend() 
pl.xlabel('redshift',fontsize=13)
pl.ylabel(cosmo.age(z).unit,fontsize=13)

pl.subplot(222)
pl.loglog(z, cosmo.comoving_distance(z) , label='Comoving distance' ) 
pl.loglog(z, cosmo.luminosity_distance(z)  , label='Luminosity distance') 
pl.loglog(z, cosmo.angular_diameter_distance(z)  , label='Angular diameter distance') 
pl.loglog(z, cosmo.lookback_distance(z) , label=r'Lookback distance' ) 
pl.xlabel('redshift',fontsize=13)
pl.ylabel(cosmo.comoving_distance(z).unit,fontsize=13)

pl.legend(fontsize=13) 
pl.subplot(224)
pl.loglog(z, cosmo.Ode(z) , label=r'$\Omega_{\Lambda}$' ) 
pl.loglog(z, cosmo.Ogamma(z) , label=r'$\Omega_{r}$' ) 
pl.loglog(z, cosmo.Om(z)  , label=r'$\Omega_{m}$') 
pl.loglog(z, cosmo.Ob(z)  , label=r'$\Omega_{b}$') 
pl.legend(fontsize=13) 
pl.xlabel('redshift',fontsize=13)


if you know a cosmological quantity and you want to know the redshift which it corresponds to, you can use `z_at_value()`.



In [ ]:
print(f"Redshift of M31 :{z_at_value(cosmo.luminosity_distance, 765*u.kpc) }" ) 
print(f"Redshift of an object in the Hubble flow :{z_at_value(cosmo.comoving_distance, cosmo.hubble_distance,) }" ) 